In [ ]:
%region us-east-1
%iam_role #setar
%idle_timeout 30
%glue_version 4.0
%worker_type G.1X
%number_of_workers 5

#%additional_python_modules s3://fiocruz-datalake-bucket/raw/aesop/dados_auxiliares/python_utils/teste.py
%extra_py_files s3://fiocruz-datalake-bucket/raw/aesop/dados_auxiliares/python_utils/util_custom.py
%additional_python_modules s3://fiocruz-datalake-bucket/raw/aesop/dados_auxiliares/epiweeks-2.3.0-py3-none-any.whl


import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

In [ ]:
from util_custom import *
from pyspark.sql import functions as F
from pyspark.sql.types import *
import os
from datetime import datetime, timedelta
from pyspark.sql.window import Window
from epiweeks import Week

import boto3
from botocore.exceptions import ClientError

def create_folder_in_specific_location(bucket_name, full_folder_path):
    # Adiciona uma barra ao final do caminho da pasta, se não houver
    if not full_folder_path.endswith('/'):
        full_folder_path += '/'
    
    s3 = boto3.resource('s3')
    bucket = s3.Bucket(bucket_name)
    
    # Verifica se a pasta já existe
    objs = list(bucket.objects.filter(Prefix=full_folder_path))
    if any(obj.key == full_folder_path for obj in objs):
        print(f"A pasta '{full_folder_path}' já existe no bucket '{bucket_name}'.")
    else:
        # Tenta criar a pasta (objeto S3 com chave terminando em '/')
        try:
            s3.Object(bucket_name, full_folder_path).put(Body=b'')
            print(f"Pasta '{full_folder_path}' criada com sucesso no bucket '{bucket_name}'.")
        except ClientError as e:
            print(f"Erro ao criar a pasta: {e}")


def list_all_files_in_folders_s3(bucket_name, prefix):
    """
    Lista todos os arquivos dentro de todas as subpastas de um prefixo específico em um bucket S3.

    :param bucket_name: Nome do bucket S3.
    :param prefix: Prefixo dentro do bucket onde as subpastas estão localizadas.
    :return: Uma lista de nomes de arquivos dentro de todas as subpastas sob o prefixo especificado.
    """
    s3 = boto3.client('s3')
    all_files = []

    # Garante que o prefixo termine com uma barra, se já não terminar.
    if not prefix.endswith('/'):
        prefix += '/'

    # Lista todas as 'subpastas' dentro do prefixo.
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix, Delimiter='/')

    subfolders = []
    if 'CommonPrefixes' in response:
        for folder in response['CommonPrefixes']:
            subfolders.append(folder['Prefix'])

    # Para cada subpasta, lista todos os arquivos dentro dela.
    for folder_name in subfolders:
        resp = s3.list_objects_v2(Bucket=bucket_name, Prefix=folder_name)
        if 'Contents' in resp:
            for file in resp['Contents']:
                # Adiciona o nome do arquivo à lista
                all_files.append(file['Key'])
    data_max = sorted(all_files)[-1][-12:-4]
    return data_max




# Exemplo de uso
bucket_name = 'fiocruz-datalake-bucket'


#### leitura banco HPC

In [ ]:
#aesop_2017_2024 = spark.read.parquet(aesop_hpc_path+get_hpc_parquet_file_name())
#aesop_hpc_path+get_hpc_parquet_file_name()
# aesop_2017_2024 = spark.read.parquet('/dados10t/datalake/standard/aesop/aesop_hpc/2017_20240107_AESOP.parquet/')

aesop_2017_2024 = spark.read.parquet("s3://fiocruz-datalake-bucket/standard/output_hpc/2017_20240331_AESOP.parquet")

In [ ]:
max_ano = aesop_2017_2024.agg(F.max("ano")).collect()[0][0]
semana_folder = aesop_2017_2024.filter(F.col("ano") == max_ano).agg(F.max("epiweek")).collect()[0][0]
semana_folder

In [ ]:
aesop_report_folder  = "standard/reports/report_dqi/"

In [ ]:
#caminho_pasta = aesop_report_folder+"semana_"+str(semana_folder)+"_"+str(max_ano)
#caminho_pasta
#create_folder_in_specific_location(bucket_name,caminho_pasta)

#### leitura banco raw parquet

In [ ]:
#df = spark.read.parquet(aesop_parquet_path+get_parquet_file_name())
#aesop_parquet_path+get_parquet_file_name()
# df = spark.read.parquet('/dados10t/datalake/raw/aesop/parquet_explorer/aesop_dados_2017_20240107.parquet/')
df = spark.read.parquet("s3://fiocruz-datalake-bucket/raw/aesop/dados-ms-parquet/aesop_dados_2017_20240331.parquet/")

In [ ]:
df.show()

In [ ]:
df.count()

In [ ]:
df = df.filter((F.col("FX_ETARIA")).isNotNull()).filter(F.col("ANO") >= 2017)
df_v2 = df.select("ANO","SEMANA","SG_UF","CO_MUNICIPIO_IBGE","SEXO","FX_ETARIA","CIDCIAP","QT","Data_folder")

In [ ]:
df_v2.count()

In [ ]:
#df_v2.filter(F.col("ANO") == 2023).show(5000,False)

#### converter cid_ciap

In [ ]:
df_v3 = df_v2\
.withColumn("Tipo_CIDCIAP", F.substring_index(df_v2.CIDCIAP, '(', 1))\
.withColumn("Codigo_CIDCIAP", F.substring_index(df_v2.CIDCIAP, '(', -1))

df_v3 = df_v3.withColumn('Codigo_CIDCIAP', F.concat(F.lit("("), F.col('Codigo_CIDCIAP')))

# CIAP(R05) -> CIAP|(R05)|

In [ ]:

# OBS: max semana folder passa da epiweek

sisab_diff = df_v3.groupBy("ANO", "SEMANA", "SG_UF", "CO_MUNICIPIO_IBGE", "Data_folder").agg(F.sum("QT").alias("qt_total")).cache()

In [ ]:
sisab_diff.count()

In [ ]:
sisab_diff.show(5)

#### gerar dados report dqi

In [ ]:
ibge_munic = spark.read.csv('s3://fiocruz-datalake-bucket/raw/aesop/dados_auxiliares/ibge_municipios.csv', header=True, sep=';').drop('_c0')

""" semanas_epis.show()
+------+----+-------+-----------------+--------------------+--------+----------+
|SEMANA| ANO|uf_ibge|CO_MUNICIPIO_IBGE|           municipio|pop_2010|     porte|
+------+----+-------+-----------------+--------------------+--------+----------+
|    23|2024|     BA|           290590|Campo Alegre de L...|   28090|Pequeno II|
|    24|2017|     BA|           290590|Campo Alegre de L...|   28090|Pequeno II|
"""

#semanas_epis = spark.read.parquet("s3://fiocruz-datalake-bucket/raw/aesop/dados_auxiliares/epiweek_ibge2017_20241231.parquet/")
semanas_epis = spark.read.csv("s3://fiocruz-datalake-bucket/raw/aesop/dados_auxiliares/epiweek_ibge2017_20241231.csv",header=True)
semanas_epis = semanas_epis.withColumnRenamed("semana_ibge", "SEMANA")
semanas_epis = semanas_epis.withColumnRenamed("ano_ibge", "ANO")
semanas_epis = semanas_epis.withColumnRenamed("co_ibge", "CO_MUNICIPIO_IBGE")



epi_week_python = spark.read.parquet("s3://fiocruz-datalake-bucket/raw/aesop/dados_auxiliares/dados_epidemiologicos_2017_2024.parquet").filter(F.col("ano_epidemiologic") <= 2024) # por dia
#epi_week_python = epi_week_python.withColumnRenamed("data", "Data_folder")

last_first_day_epiweek = spark.read.parquet("s3://fiocruz-datalake-bucket/raw/aesop/dados_auxiliares/epi_week_first_last_day_2017a2024.parquet/") # por semana com último dia de cada semana epid.


epi_week = epi_week_python.join(last_first_day_epiweek,['semana_epidemiologica','ano_epidemiologic'],'inner')

epi_week_dt_folder = epi_week.withColumnRenamed("data", "Data_folder").select("Data_folder","ultimo_dia_week_epidemio")

In [ ]:
epi_week_python.count()

In [ ]:
last_first_day_epiweek.count()

In [ ]:
epi_week.count()

In [ ]:
epi_week_dt_folder.count()

In [ ]:
epi_week_python.show(5)

In [ ]:
last_first_day_epiweek.show(5)

In [ ]:
epi_week.show(5)

In [ ]:
epi_week_dt_folder.show(5)

In [ ]:
epi_week_python.groupBy("ano_epidemiologic").agg(F.countDistinct("semana_epidemiologica")).orderBy("ano_epidemiologic").show(10)

In [ ]:
#epi_week_python = epi_week_python.withColumn("semana_epidemiologica",F.when( F.col("ano_epidemiologic") == "2021" &  F.col("semana_epidemiologica") == "53","52"  ).otherwise(F.col("semana"))

In [ ]:
sisab_diff.printSchema()

In [ ]:
epi_week_dt_folder.printSchema()

In [ ]:
sisab_diff.count()

In [ ]:
#add datas e dados a partir da var DAta Folder

sisab_diff = sisab_diff.join(epi_week_dt_folder,['Data_folder'],'inner')

#sisab_diff = sisab_diff.withColumnRenamed("semana_epidemiologica", "week_epi_Data_folder")

#sisab_diff = sisab_diff.withColumnRenamed("ano_epidemiologic", "ano_epi_Data_folder")

sisab_diff = sisab_diff.withColumnRenamed("ultimo_dia_week_epidemio", "dt_folder")

In [ ]:
sisab_diff.count()

In [ ]:
sisab_diff.show(5)

In [ ]:



#add datas e dados a partir da var semana e ano do atendimento

last_first_day_epiweek = last_first_day_epiweek.withColumnRenamed("ultimo_dia_week_epidemio", "dt_encounter").drop("primeiro_dia_week_epidemio")
last_first_day_epiweek = last_first_day_epiweek.withColumnRenamed("semana_epidemiologica", "SEMANA")
last_first_day_epiweek = last_first_day_epiweek.withColumnRenamed("ano_epidemiologic", "ANO")

sisab_diff = sisab_diff.join(last_first_day_epiweek,['SEMANA','ANO'],'inner')



#sisab_diff_v2 = sisab_diff_v2.withColumnRenamed("semana_epidemiologica", "week_epi_Data_folder")
#sisab_diff_v2 = sisab_diff_v2.withColumnRenamed("ano_epidemiologic", "ano_epi_Data_folder")

#sisab_diff_v2 = sisab_diff_v2.withColumnRenamed("primeiro_dia_week_epidemio", "first_dia_week_epidemio_dt_folder")
#sisab_diff_v2 = sisab_diff_v2.withColumnRenamed("ultimo_dia_week_epidemio", "last_dia_week_epidemio_dt_folder")
#sisab_diff_v2 = sisab_diff.join(epi_week,['Data_folder'],'inner')

In [ ]:
sisab_diff.count()

In [ ]:
sisab_diff.show(5)

In [ ]:
teste = sisab_diff.join(last_first_day_epiweek,['SEMANA','ANO'],'leftanti')
print(teste.show(30))
teste.count()

In [ ]:
#diferença em dias de dt_folder para dt_encounter
sisab_diff = sisab_diff.withColumn("date_diff", F.datediff("dt_folder","dt_encounter") ) 

#diff de semanas entre atendimento estimado e data do envio
sisab_diff = sisab_diff.withColumn("week_diff", F.col("date_diff") / 7 )  

In [ ]:
sisab_diff.show(5)

In [ ]:
#sisab_diff.filter(F.col("ANO") == 2019).filter(F.col("SEMANA") == 52).show(100)

#### calcular filtros de 8 semanas

In [ ]:
#semana_folder

In [ ]:
semana_filter = semana_folder-7
ano_filter = max_ano
if semana_filter < 1:
    ano_filter = ano_filter-1
    semana_filter = 52+semana_filter
    
print(semana_filter)
print(ano_filter)

In [ ]:
semana_folder

In [ ]:
#sisab_diff.filter(F.col('ANO') == ano_filter).filter(F.col('SEMANA') == semana_filter).select("dt_encounter").distinct().count() #aqui tem somente 1 data

In [ ]:
filter_date = sisab_diff.filter(F.col('ANO') == ano_filter)\
.filter(F.col('SEMANA') == semana_filter)\
.select(F.max(F.col('dt_encounter')) ).collect()[0][0]
print(filter_date) #2024-02-17 para semana 14 usando o sabado

In [ ]:
sisab_diff.filter(sisab_diff['dt_encounter'] >= filter_date).select("SEMANA").distinct().count()

In [ ]:
#completude.printSchema()

In [ ]:
completude = sisab_diff.filter(sisab_diff['dt_encounter'] >= filter_date)

In [ ]:
completude.count() #244260

In [ ]:
completude.printSchema()

In [ ]:
completude = completude\
.drop('Data_folder', 'ANO', 'SEMANA', 'SG_UF', 'first_day_of_year', 'date_diff')\
.withColumnRenamed('CO_MUNICIPIO_IBGE', 'co_ibge')

In [ ]:
completude.printSchema()

In [ ]:
ibge = completude.select('co_ibge').distinct()

In [ ]:
ibge.count()

In [ ]:
dt_encounter = completude.select('dt_encounter').distinct()


In [ ]:
dt_encounter.show()

In [ ]:
dt_encounter.count()

In [ ]:
ibge_dtencounter = ibge.crossJoin(dt_encounter)

In [ ]:
ibge_dtencounter.count()

In [ ]:
ibge_dtencounter.filter(F.col("co_ibge") == "350510").show(30,False)

In [ ]:
completude.printSchema()

In [ ]:
completude = ibge_dtencounter.join(completude, ['co_ibge', 'dt_encounter'], 'left')

In [ ]:
completude.count() #251358

In [ ]:
completude.filter(F.col("co_ibge") == "355580").orderBy("dt_encounter").show(30,False)

In [ ]:
completude.show(5,False)

In [ ]:
completude_grouped = completude.groupBy('co_ibge', 'dt_encounter')\
.agg(F.sum('qt_total').alias('n'))\
.na.fill({'n': 0})

In [ ]:
completude_grouped.filter(F.col("co_ibge") == "355580").orderBy("dt_encounter").show(30,False)

In [ ]:
completude_grouped.show(5)

#### completude

In [ ]:
# Assuming `df` is your Spark DataFrame equivalent to `sisab_diff`
# and `week_filter` is already defined

#week_filter = "2023-01-07"

# Step 1: Filter based on `dt_folder`
completude = sisab_diff.filter(sisab_diff['dt_encounter'] >= filter_date)

# Step 2: Select specific columns and remove others
#tempestividade = tempestividade.drop('Data_folder', 'ANO', 'SEMANA', 'SG_UF')
completude = completude\
.drop('Data_folder', 'ANO', 'SEMANA', 'SG_UF', 'first_day_of_year', 'date_diff')\
.withColumnRenamed('CO_MUNICIPIO_IBGE', 'co_ibge')

# complete command
ibge = completude.select('co_ibge').distinct()
dt_encounter = completude.select('dt_encounter').distinct()
ibge_dtencounter = ibge.crossJoin(dt_encounter)
# complete join back
completude = ibge_dtencounter.join(completude, ['co_ibge', 'dt_encounter'], 'left')

"""
completude.show()
+-------+------------+----+
|co_ibge|dt_encounter|   n|
+-------+------------+----+
| 150375|  2023-11-25| 351|
| 150375|  2023-11-18| 336|
"""

# count 
completude = completude.groupBy('co_ibge', 'dt_encounter')\
.agg(F.sum('qt_total').alias('n'))\
.na.fill({'n': 0})

In [ ]:
semana_folder

#### completude porcentagem

In [ ]:
ibge_munic.printSchema()

In [ ]:
# slider
ws = Window.partitionBy("co_ibge").orderBy("dt_encounter").rowsBetween(-7, Window.currentRow)

# UDF for epiweek and epiyear
epiweekUDF = F.udf(lambda z:  Week.fromdate(z).week, IntegerType())
epiyearUDF = F.udf(lambda z:  Week.fromdate(z).year, IntegerType())

# slider as window function
completude_missings = completude\
.withColumn("empty", F.when(F.col('n')==0, 1).otherwise(0))\
.withColumn("window_size", F.count("*").over(ws))\
.withColumn("start", F.min("dt_encounter").over(ws))\
.withColumn("end", F.max("dt_encounter").over(ws))\
.withColumn("no_missings", F.sum("empty").over(ws))\
.filter(F.col('window_size')==8)\
.withColumn("epi_week", epiweekUDF(F.col('dt_encounter')))\
.withColumn("epi_year", epiyearUDF(F.col('dt_encounter')))\
.drop('window_size', 'empty', 'n', 'dt_encounter')\
.filter(F.col('epi_week')==semana_folder)

# add missing cities 
completude_miss_ibge = ibge_munic.join(completude_missings, ['co_ibge'], 'full')

# perc completude
"""
+-------+---+--------------------+--------+----------+----------+----------+-----------+--------+--------+------------+---------------+
|co_ibge| uf|           municipio|pop_2010|     porte|     start|       end|no_missings|epi_week|epi_year|perc_missing|perc_completude|
+-------+---+--------------------+--------+----------+----------+----------+-----------+--------+--------+------------+---------------+
| 110001| RO|Alta Floresta D´o...|   24392|Pequeno II|2023-11-11|2023-12-30|          0|      52|    2023|         0.0|          100.0|
| 110002| RO|           Ariquemes|   90353|     Médio|2023-11-11|2023-12-30|          1|      52|    2023|        12.5|           87.5|
"""

# calculate percentages
comp_percentage = completude_miss_ibge.na.fill({'no_missings': 8})\
.withColumn('perc_missing',  F.col('no_missings')/8*100)\
.withColumn('perc_completude',  100-F.col('perc_missing'))




In [ ]:
semana_filter

In [ ]:
filter_date

In [ ]:
sisab_diff.filter(sisab_diff['dt_folder'] >= filter_date).select("SEMANA").distinct().count()

In [ ]:
sisab_diff.filter(sisab_diff['dt_folder'] >= filter_date).select("dt_folder").distinct().count()

In [ ]:
tempestividade = sisab_diff.filter( F.col("dt_folder")  >= filter_date)

# Step 2:dropa algumas colunas
tempestividade = tempestividade.drop( 'SG_UF') #, 'ANO', 'SEMANA','Data_folder',

In [ ]:
df_v3.printSchema()

In [ ]:
df_v3.filter(F.col("CO_MUNICIPIO_IBGE") == "354810").orderBy(F.desc("Data_folder")).show(30)

In [ ]:
tempestividade.filter(F.col("CO_MUNICIPIO_IBGE") == "354810").orderBy(F.desc("dt_folder")).show(30,False) # dt folder é repetido pois são de semanas/ano diferentes # 355580
#+----+------+-----------------+-----------+--------+-----------------+------------+----------+---------+---------+
#|ANO |SEMANA|CO_MUNICIPIO_IBGE|Data_folder|qt_total|first_day_of_year|dt_encounter|dt_folder |date_diff|week_diff|
#+----+------+-----------------+-----------+--------+-----------------+------------+----------+---------+---------+
#|2024|13    |354810           |2024-04-07 |584     |2024-01-01       |2024-03-30  |2024-04-13|14       |2.0      |
#|2024|12    |354810           |2024-04-07 |701     |2024-01-01       |2024-03-23  |2024-04-13|21       |3.0      |
#|2024|11    |354810           |2024-04-07 |661     |2024-01-01       |2024-03-16  |2024-04-13|28       |4.0      |
#|2024|14    |354810           |2024-04-07 |36      |2024-01-01       |2024-04-06  |2024-04-13|7        |1.0      |

In [ ]:
tempestividade.printSchema()

In [ ]:
co_ibge_df = tempestividade.select("CO_MUNICIPIO_IBGE").distinct()
dt_folder_df = tempestividade.select("dt_folder").distinct()
all_combinations_df = co_ibge_df.crossJoin(dt_folder_df)


In [ ]:
co_ibge_df.count()

In [ ]:
dt_folder_df.count()

In [ ]:
dt_folder_df.orderBy("dt_folder").show(10)

In [ ]:
all_combinations_df.count()

In [ ]:
all_combinations_df.filter(F.col("CO_MUNICIPIO_IBGE") == "355580").orderBy("dt_folder").show(30,False)

In [ ]:
tempestividade.count()

In [ ]:
tempestividade.show(10)

In [ ]:
all_combinations_df.count()

In [ ]:
tempestividade.count()

In [ ]:
# faz o join para o lado completo criando anteriormente
tempestividade2 = all_combinations_df.join(tempestividade, ["CO_MUNICIPIO_IBGE", "dt_folder"], "left_outer")

In [ ]:
tempestividade2.count() #345165

In [ ]:
tempestividade2.filter(F.col("dt_folder") == "2024-04-13").count()

In [ ]:
tempestividade2.filter(F.col("dt_folder") == "2024-04-13").filter(F.col("qt_total").isNull()).count()

In [ ]:
tempestividade2.filter(F.col("dt_folder") == "2024-04-13").filter(F.col("qt_total").isNotNull()).show(10)

In [ ]:
tempestividade2.filter(F.col("CO_MUNICIPIO_IBGE") == "110140").agg(F.sum("qt_total")).collect()[0][0]

In [ ]:
tempestividade2.filter(F.col("CO_MUNICIPIO_IBGE") == "110140").select("dt_folder").distinct().count()

In [ ]:
tempestividade2.filter(F.col("CO_MUNICIPIO_IBGE") == "110140").orderBy("dt_folder").show(80,False) # dt_folder aqui e o agrupamento dos datafolder para o ultimo dia da semana epi por isso temos repeticao nessa e nao tem na var 
#Data_folder que vem da pasta

In [ ]:
sisab_diff.filter(F.col("CO_MUNICIPIO_IBGE") == "110140").filter(F.col("dt_folder") >= "2024-02-17").orderBy("dt_folder").show(100)

In [ ]:
tempestividade3_p1 = tempestividade2.groupBy("CO_MUNICIPIO_IBGE").agg(F.sum("qt_total").alias("total_atd_folder"))

In [ ]:
tempestividade3_p1.show(5)

In [ ]:
tempestividade3_p1.filter(F.col("CO_MUNICIPIO_IBGE") == "110140").show(5)

In [ ]:
tempestividade2 = tempestividade2.withColumn("diff", F.when(  F.col("week_diff") <= 2, "0_2weeks").otherwise(
                                                        F.when(   F.col("week_diff") > 2, "3+weeks"  )    ) )

In [ ]:
tempestividade2.filter(F.col("CO_MUNICIPIO_IBGE") == "110140").orderBy(F.desc("dt_folder")).show(60,False) #355580

In [ ]:
#faz categorias  a partir da diff em semanas entre atendimento e envio para as ultimas 8 semanas. se ate 2 semanas de diff,  0_2weeks , se maior que duas semanas, 3+weeks,soma-se o total de atendimento para cada categoria criada.
tempestividade3_p2 = tempestividade2.groupBy("CO_MUNICIPIO_IBGE","diff").agg( F.sum("qt_total").alias("qt_diff") )

In [ ]:
tempestividade3_p2.filter(F.col("CO_MUNICIPIO_IBGE") == "110140").show(30,False)

In [ ]:
tempestividade3_p2.show(5)

In [ ]:
tempestividade3 = tempestividade3_p1.join(tempestividade3_p2 , ['CO_MUNICIPIO_IBGE'])
tempestividade3 = tempestividade3.withColumn("diff_percent", F.col("qt_diff") / F.col("total_atd_folder") * 100)
tempestividade3 = tempestividade3.withColumn("diff_percent2",F.round("diff_percent",5))

In [ ]:
tempestividade3.filter(F.col("CO_MUNICIPIO_IBGE") == "110140").show(30,False)

In [ ]:
#pivota os dados para as variáveis de porcetagem 0_2weeks e 3+weeks irem para colunas
tempestividade4  = tempestividade3.groupBy("CO_MUNICIPIO_IBGE").pivot("diff").agg(F.first("diff_percent2"))


tempestividade5 = tempestividade4.withColumnRenamed("0_2weeks","diff_2w")
tempestividade5 = tempestividade5.withColumnRenamed("3+weeks","diff_3w")
tempestividade5 = tempestividade5.withColumn("co_ibge",F.col("CO_MUNICIPIO_IBGE").cast(StringType() ) ).drop("CO_MUNICIPIO_IBGE","diff_2w_new","diff_3w_new","null").cache()

In [ ]:
tempestividade5.filter(F.col("CO_MUNICIPIO_IBGE") == "110140").show(30,False)

#### tempestividade 

In [ ]:
# Step 1: Filter based on `dt_folder`
tempestividade = sisab_diff.filter( F.col("dt_folder")  >= filter_date)

# Step 2:dropa algumas colunas
tempestividade = tempestividade.drop('Data_folder', 'ANO', 'SEMANA', 'SG_UF')

# faz o complete dos dados levando em conta todas as possibilidades de CO_MUNICIPIO_IBGE x dt_folder
co_ibge_df = tempestividade.select("CO_MUNICIPIO_IBGE").distinct()
dt_folder_df = tempestividade.select("dt_folder").distinct()
all_combinations_df = co_ibge_df.crossJoin(dt_folder_df)

# faz o join para o lado completo criando anteriormente
tempestividade2 = all_combinations_df.join(tempestividade, ["CO_MUNICIPIO_IBGE", "dt_folder"], "left_outer")

#conta os atendimento para as 8 semanas de dados
tempestividade3_p1 = tempestividade2.groupBy("CO_MUNICIPIO_IBGE").agg(F.sum("qt_total").alias("total_atd_folder"))

#faz categorias  a partir da diff em semanas entre atendimento e envio para as ultimas 8 semanas. se ate 2 semanas de diff,  0_2weeks , se maior que duas semanas, 3+weeks,soma-se o total de atendimento para cada categoria criada.
tempestividade3_p2 = tempestividade2.withColumn("diff", F.when(  F.col("week_diff") <= 2, "0_2weeks").otherwise(
                                                        F.when(   F.col("week_diff") > 2, "3+weeks"  )    ) ) .groupBy("CO_MUNICIPIO_IBGE","diff").agg( F.sum("qt_total").alias("qt_diff") )

#junta o atendimento total das 8 semanas com os atendimentos das 8 semanas categorizado entre 0_2weeks e   3+weeks
tempestividade3 = tempestividade3_p1.join(tempestividade3_p2 , ['CO_MUNICIPIO_IBGE'])
tempestividade3 = tempestividade3.withColumn("diff_percent", F.col("qt_diff") / F.col("total_atd_folder") * 100)
tempestividade3 = tempestividade3.withColumn("diff_percent2",F.round("diff_percent",5))

#pivota os dados para as variáveis de porcetagem 0_2weeks e 3+weeks irem para colunas
tempestividade4  = tempestividade3.groupBy("CO_MUNICIPIO_IBGE").pivot("diff").agg(F.first("diff_percent2"))

""" tempestividade5
+--------+--------+-------+
| diff_2w| diff_3w|co_ibge|
+--------+--------+-------+
|99.83987| 0.16013| 150300|
|54.82866|45.17134| 150375|
|   100.0|    null| 172015|
"""

tempestividade5 = tempestividade4.withColumnRenamed("0_2weeks","diff_2w")
tempestividade5 = tempestividade5.withColumnRenamed("3+weeks","diff_3w")
tempestividade5 = tempestividade5.withColumn("co_ibge",F.col("CO_MUNICIPIO_IBGE").cast(StringType() ) ).drop("CO_MUNICIPIO_IBGE","diff_2w_new","diff_3w_new","null").cache()

In [ ]:
df_agg = sisab_diff.groupBy("ANO","SEMANA","SG_UF","CO_MUNICIPIO_IBGE").agg(F.sum("qt_total").alias("qt_total")) 

In [ ]:
df_agg.filter(F.col("CO_MUNICIPIO_IBGE").isNull() ).count()

In [ ]:
df_agg.filter(F.col("qt_total").isNull() ).count()

In [ ]:
df_agg.groupBy("ano").agg(F.countDistinct("semana")).show(10)

In [ ]:
df_agg.show(5)

In [ ]:
semanas_epis.printSchema()

In [ ]:
epi_week.printSchema()

In [ ]:
co_ibge_unicos = semanas_epis.select("CO_MUNICIPIO_IBGE").distinct()

In [ ]:
co_ibge_unicos.count()

In [ ]:
semana_folder

In [ ]:
epi_week_atend_2week = epi_week.withColumnRenamed('semana_epidemiologica', 'SEMANA')
epi_week_atend_2week = epi_week_atend_2week.withColumnRenamed('ano_epidemiologic', 'ANO')
epi_week_atend_2week = epi_week_atend_2week.select("SEMANA","ANO").dropDuplicates()


epi_week_atend_2week_v2 = epi_week_atend_2week.filter(
    (F.col("ANO").between(2017, 2023)) |
    ((F.col("ANO") == 2024) & (F.col("SEMANA") <= 13))
)

epi_week_atend_2week_v3 = epi_week_atend_2week_v2.crossJoin(co_ibge_unicos)

In [ ]:
epi_week_atend_2week_v3.count()

In [ ]:
#epi_week_atend_2week_v3.groupBy("ANO","SEMANA").agg( F.countDistinct("CO_MUNICIPIO_IBGE") ).orderBy("ano","SEMANA") .show(1000) # ok tem todos os municipios 5570 para todas as semanas de 2017 ate a semana_folder de 2024(teste para 13)

In [ ]:
epi_week_atend_2week_v3.show(10)

In [ ]:
#coount_Antiga_week = spark.read.parquet("s3://fiocruz-datalake-bucket/raw/aesop/dados_auxiliares/day_ibge.parquet")
#coount_Antiga_week.groupBy("ano","semana").agg( F.countDistinct("co_ibge") ).orderBy("ano","semana") .show(1000)

In [ ]:
semanas_epis.count()

In [ ]:
epi_week_atend_2week_v3.printSchema()

In [ ]:
#df_agg.groupBy("ANO","SEMANA").agg( F.countDistinct("CO_MUNICIPIO_IBGE") ).orderBy("ano","SEMANA") .show(1000) para cada semana nao temos aqui todos os municipios

In [ ]:
test_novo = epi_week_atend_2week_v3.join(df_agg , ["ANO","SEMANA","CO_MUNICIPIO_IBGE"] , 'left').cache()
test_novo_inner = epi_week_atend_2week_v3.join(df_agg , ["ANO","SEMANA","CO_MUNICIPIO_IBGE"] , 'inner')

In [ ]:
test_novo.count()

In [ ]:
test_novo_inner.count()

In [ ]:
2105460-2047598

In [ ]:
#test_novo.groupBy("ANO","SEMANA").agg( F.countDistinct("CO_MUNICIPIO_IBGE") ).orderBy("ano","SEMANA") .show(1000)

In [ ]:
test_novo.groupBy("ANO").agg(F.countDistinct("SEMANA")).orderBy("ano").show(10)

In [ ]:
df_agg_v2 = semanas_epis.join(df_agg , ["ANO","SEMANA","CO_MUNICIPIO_IBGE"] , 'left')

In [ ]:
teste123 = semanas_epis.join(df_agg , ["ANO","SEMANA","CO_MUNICIPIO_IBGE"] , 'inner')

In [ ]:
df_agg_v2.count()

In [ ]:
test_novo.show(10)

In [ ]:
#df_agg_v2.show(10)

In [ ]:
2328260-14333459

In [ ]:
check_anti = df_agg_v2.join(test_novo,["ANO","SEMANA","CO_MUNICIPIO_IBGE"],'leftanti')

In [ ]:
check_anti2 = test_novo.join(df_agg_v2,["ANO","SEMANA","CO_MUNICIPIO_IBGE"],'leftanti')

In [ ]:
check_anti.count()

In [ ]:
check_anti2.count()

In [ ]:
#check_anti.show(3000)

In [ ]:
cont_semana_antigo = semanas_epis.select("semana","ano").dropDuplicates().withColumn("semana",F.col("semana").cast(IntegerType()))

In [ ]:
cont_semana_antigo.count()

In [ ]:
cont_semana_antigo.groupBy("ano").agg(F.countDistinct("semana")).orderBy("ano").show(10)

In [ ]:
max_ano

In [ ]:
#junta com os dados das semanas epidemiologicas completas, filtrando em seguida para dados ate 2023 e que a semana sejam somente as duas últimas, ou seja menor que a semana folder(semana atual) e maior que a semana folder - 1, ex: semana 47 seria de 46 a 47
df_agg_v2 = epi_week_atend_2week_v3.join(df_agg , ["ANO","SEMANA","CO_MUNICIPIO_IBGE"] , 'left')
#df_agg_v3 = df_agg_v2.filter(F.col("ANO") == 2024)


if semana_folder >= 2: # para 2 pega 1 e 2, para 5 para 5 e 4 ,e assim por diante
    df_agg_v4 = df_agg_v2.filter(F.col("SEMANA") <= semana_folder).filter(F.col("SEMANA") >= semana_folder-1).filter(F.col("ANO") == max_ano)
elif semana_folder ==1:
    if max_ano == 2022:
        df_agg_v4_p1 = df_agg_v2.filter(F.col("SEMANA") <= semana_folder)  #.filter(F.col("SEMANA") >= semana_folder-1)
        df_agg_v4_p2 = df_agg_v2.filter(F.col("SEMANA") == 53).filter(F.col("ANO") == 2021)
        df_agg_v4 = df_agg_v4_p1.unionByName(df_agg_v4_p2)
    else:
        df_agg_v4_p1 = df_agg_v2.filter(F.col("SEMANA") <= semana_folder)  
        df_agg_v4_p2 = df_agg_v2.filter(F.col("SEMANA") == 52).filter(F.col("ANO") == max_ano-1)
        df_agg_v4 = df_agg_v4_p1.unionByName(df_agg_v4_p2) 
    

In [ ]:
df_agg_v4.filter(F.col("CO_MUNICIPIO_IBGE") == "110140").show(30,False)

In [ ]:
df_agg_v4.filter(F.col("CO_MUNICIPIO_IBGE") == "110140").show(30,False)

In [ ]:
#df_agg_v4.show(10)

In [ ]:
windowSpec = Window.partitionBy("CO_MUNICIPIO_IBGE")

In [ ]:


df_agg_v5 = df_agg_v4.withColumn("count_empty", F.sum(F.when(F.col("qt_total").isNull(), 1).otherwise(0)).over(windowSpec))
df_agg_v6 = df_agg_v5.select("CO_MUNICIPIO_IBGE","SEMANA","ANO","count_empty").groupBy("CO_MUNICIPIO_IBGE").agg(F.first("count_empty").alias("sum_miss"))
df_agg_v7 = df_agg_v6.withColumn("co_ibge",F.col("CO_MUNICIPIO_IBGE").cast(IntegerType()).cast(StringType()) ).drop("CO_MUNICIPIO_IBGE")



In [ ]:
df_agg_v5.filter(F.col("CO_MUNICIPIO_IBGE") == "110140").show(100,False)

In [ ]:
df_agg_v6.filter(F.col("CO_MUNICIPIO_IBGE") == "110140").show(100,False)

In [ ]:
#df_agg_v5.filter(F.col("CO_MUNICIPIO_IBGE") == "110140").show(30,False)

#### Atendimentos (atd) nas últimas semanas
TODO: verificar filtro ano

In [ ]:
#agrupar os dados de "ANO","SEMANA","SG_UF","CO_MUNICIPIO_IBGE" somando o qt_total
df_agg = sisab_diff.groupBy("ANO","SEMANA","SG_UF","CO_MUNICIPIO_IBGE").agg(F.sum("qt_total").alias("qt_total")) 

#junta com os dados das semanas epidemiologicas completas, filtrando em seguida para dados ate 2023 e que a semana sejam somente as duas últimas, ou seja menor que a semana folder(semana atual) e maior que a semana folder - 1, ex: semana 47 seria de 46 a 47
df_agg_v2 = epi_week_atend_2week_v3.join(df_agg , ["ANO","SEMANA","CO_MUNICIPIO_IBGE"] , 'left')

if semana_folder >= 2: # para 2 pega 1 e 2, para 5 para 5 e 4 ,e assim por diante
    df_agg_v4 = df_agg_v2.filter(F.col("SEMANA") <= semana_folder).filter(F.col("SEMANA") >= semana_folder-1).filter(F.col("ANO") == max_ano)
elif semana_folder ==1:
    if max_ano == 2022:
        df_agg_v4_p1 = df_agg_v2.filter(F.col("SEMANA") <= semana_folder)  #.filter(F.col("SEMANA") >= semana_folder-1)
        df_agg_v4_p2 = df_agg_v2.filter(F.col("SEMANA") == 53).filter(F.col("ANO") == 2021)
        df_agg_v4 = df_agg_v4_p1.unionByName(df_agg_v4_p2)
    else:
        df_agg_v4_p1 = df_agg_v2.filter(F.col("SEMANA") <= semana_folder)  
        df_agg_v4_p2 = df_agg_v2.filter(F.col("SEMANA") == 52).filter(F.col("ANO") == max_ano-1)
        df_agg_v4 = df_agg_v4_p1.unionByName(df_agg_v4_p2) 

#para criar a variável sum_miss que tem 3 categorias, 0(com atendimentos nas 2 semanas) , 1 (com atendimento em somente 1 das duas semanas ) e 2 (sem atendimetnos nas duas semanas)
windowSpec = Window.partitionBy("CO_MUNICIPIO_IBGE")

df_agg_v5 = df_agg_v4.withColumn("count_empty", F.sum(F.when(F.col("qt_total").isNull(), 1).otherwise(0)).over(windowSpec))
df_agg_v6 = df_agg_v5.select("CO_MUNICIPIO_IBGE","SEMANA","ANO","count_empty").groupBy("CO_MUNICIPIO_IBGE").agg(F.first("count_empty").alias("sum_miss"))
df_agg_v7 = df_agg_v6.withColumn("co_ibge",F.col("CO_MUNICIPIO_IBGE").cast(IntegerType()).cast(StringType()) ).drop("CO_MUNICIPIO_IBGE")

dqi = df_agg_v7.join(tempestividade5,['co_ibge'],'left')

In [ ]:
dqi.filter(F.col("co_ibge") == "110140").show(100,False)

In [ ]:
dqi_v2 = dqi.join(comp_percentage,['co_ibge'],'left')

dqi_v2 = dqi_v2.withColumn("tempestividade",F.when(F.col("diff_2w") >=  75,  0).otherwise(1)  )
dqi_v2 = dqi_v2.withColumn("completude",F.when(F.col("perc_completude") >=  85, 0).otherwise(1)  )

cond1 = F.col("completude") == 0
cond2 = F.col("tempestividade") == 0
cond3 = F.col("sum_miss") == 0

dqi_v2 = dqi_v2.withColumn("dqi",F.when( cond1 & cond2 & cond3, "Apto").otherwise("Não Apto")  )

In [ ]:
""" dqi_v3.show()
+---+-------+-------------------+--------+--------+------------+---------------+--------+--------+--------+----------+--------------+--------+
| uf|co_ibge|          municipio|epi_week|epi_year|perc_missing|perc_completude| diff_2w| diff_3w|sum_miss|completude|tempestividade|     dqi|
+---+-------+-------------------+--------+--------+------------+---------------+--------+--------+--------+----------+--------------+--------+
| PA| 150200| Cachoeira do Arari|      52|    2023|        12.5|           87.5|32.79879|67.20121|       1|         0|             1|Não Apto|
| PI| 220290|           Corrente|      52|    2023|         0.0|          100.0| 91.6261|  8.3739|       0|         0|             0|    Apto|
"""

dqi_v3 = dqi_v2.select("uf", "co_ibge", "municipio", "epi_week", "epi_year", "perc_missing", "perc_completude", "diff_2w", "diff_3w", "sum_miss", "completude", "tempestividade", "dqi")
dqi_v3 = dqi_v3.na.fill({'diff_2w': 0, 'diff_3w': 0})

In [ ]:
caminho_pasta

In [ ]:
caminho_pasta

In [ ]:
#path_salvar = "s3://fiocruz-datalake-bucket/"+caminho_pasta
path_salvar = "s3://fiocruz-datalake-bucket/standard/reports/report_dqi/dqi_retrospectivo/"

In [ ]:
#path_salvar_dqi = aesop_dqi_report_folder+'semana_'+str(semana_folder)+"/dqi.csv"


dqi_v3.toPandas().to_csv(path_salvar+"dqi_13_teste.csv",index=False)
path_salvar

In [ ]:
%stop_session